## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using StatsBase
using Distributions

In [2]:
using TensorFlow
using MLDataUtils

In [3]:
using DensityEstimationML

In [4]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    conditioner::Tensor
    t::Tensor
    pdf::Tensor
end

In [5]:
function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    gr = est.sess.graph
    ts = reshape([t], (1,1))
    run(est.sess, est.pdf, Dict(est.t=>ts))[1]
end

In [6]:
function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    epochs = 20)
    
    gr = estimator.sess.graph
    for ii in 1:epochs
        loss_o = run(estimator.sess, 
            [gr["ysmin"],gr["ysmax"],gr["true_loss"], gr["working_loss"],
                estimator.optimizer],
            Dict(estimator.t=>observations'))
        ii % 100 == 1 && println("Epoch $ii: loss: $(loss_o)")
    end
    estimator
end

In [9]:
function NeuralDensityEstimator(prob_layer_sizes, support_min, support_max)
    sess = Session(Graph())
    @tf begin
        t = placeholder(Float32, shape=[1, -1])
        smin = constant(reshape([support_min],(1,1)))
        smax = constant(reshape([support_max],(1,1)))
        
        layer_sizes= [1; prob_layer_sizes; 1]
        
        network_fun_stack = Function[Base.identity]       
       
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
                       
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            Wii2  = Ops.mul(Wii, Wii; name = "W_$(ii)_squared")
            act_fun = if ii!=length(layer_sizes)
                bii = get_variable("b_$ii", [above_size, 1], Float32)
                z -> nn.sigmoid(Wii2*z .+ bii)
            else
                z-> exp(Wii2*z)
            end
            push!(network_fun_stack, z->act_fun(network_fun_stack[ii-1](z)))
        end
        
        network = network_fun_stack[end]
        
        #zsmin = network(smin)
        #zsmax = network(smax)
        
        #ysmin = min(zsmin,zsmax)
        #ysmax = max(zsmin,zsmax)
        #yt = select(zsmin[end]<zsmax[end], network(t), network(-1.*t))
        
        ysmin = TensorFlow.identity(network(smin))
        ysmax = TensorFlow.identity(network(smax))
        yt = network(t)
        
        denominator = (ysmax-ysmin) #area
        numerator = gradients(yt,t)
        pdf =numerator/denominator
        
        
        n_points = TensorFlow.shape(t)[2]
        true_loss= -reduce_sum(log(numerator))+ n_points.*log(denominator)
        
        area_loss = (1f0.-denominator)^2
        working_loss = TensorFlow.identity(true_loss) + 0.1*area_loss
        
        optimizer = train.minimize(train.AdamOptimizer(), working_loss)
        
        
        # Conditioning
        # Make sure that ysmin~=1, and ysmax~=2
        condition_loss = (1f0 - ysmin)^2 + (2f0 - ysmax)^2
        condition_optimiser = train.minimize(train.AdamOptimizer(;name="adam_cond"), condition_loss)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, condition_optimiser, t, pdf)
end

NeuralDensityEstimator

In [11]:
"""
    condition(est::NeuralDensityEstimator tol = 1e-15, max_epochs=2_000)
    
"Conditions" the neural density estimate so the support extrema are mapped to 1. and 2.
This improves training by adjusting the area the network has the learn over

"""
function condition!(est::NeuralDensityEstimator, tol = 1e-15, max_epochs=2_000)
    gr = est.sess.graph
    for ii in 1:2_000
        _, ysmin, ysmax, condition_loss = run(est.sess, [est.conditioner, gr["ysmin"],gr["ysmax"], gr["condition_loss"]])
        ii % 50 == 1 && @show (ii, ysmin, ysmax, condition_loss)
        if condition_loss[1] < 1e-15
            break
        end
    end
end


condition!

In [14]:
dataset = GenerateDatasets.likas_1
data = dataset()
est = NeuralDensityEstimator([10,10], support(dataset)...)


2017-09-08 18:31:41.056072: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


NeuralDensityEstimator(Session(Ptr{Void} @0x00007f4998ab07b0), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [15]:
condition!(est)
println("Conditioning Done")
fit!(est, data; epochs=20_000)

(ii, ysmin, ysmax, condition_loss) = (1, [1.00027], [1.00027], [0.999458])
(ii, ysmin, ysmax, condition_loss) = (51, [1.02225], [1.02225], [0.956487])
(ii, ysmin, ysmax, condition_loss) = (101, [1.11846], [1.12004], [0.788366])
(ii, ysmin, ysmax, condition_loss) = (151, [1.33488], [1.37219], [0.506289])
(ii, ysmin, ysmax, condition_loss) = (201, [1.48697], [1.66436], [0.349793])
(ii, ysmin, ysmax, condition_loss) = (251, [1.45201], [1.74686], [0.268391])
(ii, ysmin, ysmax, condition_loss) = (301, [1.42323], [1.79879], [0.219613])
(ii, ysmin, ysmax, condition_loss) = (351, [1.40022], [1.82891], [0.189446])
(ii, ysmin, ysmax, condition_loss) = (401, [1.38188], [1.84947], [0.168495])
(ii, ysmin, ysmax, condition_loss) = (451, [1.36648], [1.86496], [0.152543])
(ii, ysmin, ysmax, condition_loss) = (501, [1.35297], [1.87739], [0.139623])
(ii, ysmin, ysmax, condition_loss) = (551, [1.34078], [1.8878], [0.128722])
(ii, ysmin, ysmax, condition_loss) = (601, [1.32956], [1.89674], [0.119271])
(ii

Epoch 6801: loss: Any[[1.19492], [6.50007], [13134.0], [13135.8], nothing]
Epoch 6901: loss: Any[[1.19259], [6.48462], [13131.5], [13133.4], nothing]
Epoch 7001: loss: Any[[1.18905], [6.4726], [13129.2], [13131.0], nothing]
Epoch 7101: loss: Any[[1.18426], [6.46235], [13126.9], [13128.7], nothing]
Epoch 7201: loss: Any[[1.17825], [6.4523], [13124.7], [13126.6], nothing]
Epoch 7301: loss: Any[[1.17107], [6.44105], [13122.7], [13124.5], nothing]
Epoch 7401: loss: Any[[1.16285], [6.42733], [13120.6], [13122.4], nothing]
Epoch 7501: loss: Any[[1.15371], [6.41009], [13118.6], [13120.5], nothing]
Epoch 7601: loss: Any[[1.14385], [6.3885], [13116.7], [13118.5], nothing]
Epoch 7701: loss: Any[[1.13349], [6.36195], [13114.8], [13116.5], nothing]
Epoch 7801: loss: Any[[1.12283], [6.32992], [13112.8], [13114.6], nothing]
Epoch 7901: loss: Any[[1.11211], [6.29202], [13110.9], [13112.7], nothing]
Epoch 8001: loss: Any[[1.10152], [6.24806], [13109.0], [13110.8], nothing]
Epoch 8101: loss: Any[[1.091

Epoch 17701: loss: Any[[1.0], [1.26199], [13012.7], [13012.7], nothing]
Epoch 17801: loss: Any[[1.0], [1.25701], [13012.4], [13012.5], nothing]
Epoch 17901: loss: Any[[1.0], [1.25222], [13012.2], [13012.2], nothing]
Epoch 18001: loss: Any[[1.0], [1.24755], [13011.9], [13012.0], nothing]
Epoch 18101: loss: Any[[1.0], [1.24309], [13011.7], [13011.7], nothing]
Epoch 18201: loss: Any[[1.0], [1.23878], [13011.4], [13011.5], nothing]
Epoch 18301: loss: Any[[1.0], [1.2346], [13011.2], [13011.3], nothing]
Epoch 18401: loss: Any[[1.0], [1.23052], [13011.0], [13011.0], nothing]
Epoch 18501: loss: Any[[1.0], [1.22661], [13010.7], [13010.8], nothing]
Epoch 18601: loss: Any[[1.0], [1.22285], [13010.5], [13010.6], nothing]
Epoch 18701: loss: Any[[1.0], [1.21919], [13010.3], [13010.3], nothing]
Epoch 18801: loss: Any[[1.0], [1.21564], [13010.1], [13010.1], nothing]
Epoch 18901: loss: Any[[1.0], [1.21225], [13009.8], [13009.9], nothing]
Epoch 19001: loss: Any[[1.0], [1.20885], [13009.6], [13009.7], no

NeuralDensityEstimator(Session(Ptr{Void} @0x00007f4998ab07b0), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [16]:
using Plots

using MacroTools
macro plot(ex)
    @capture(ex, (x_, [ys__], tail__ )) 
    labels = repr.(ys)
    ys_expr = Expr(:hvect, ys...)
    labels_expr = Expr(:vect, repr.(ys)...)
    Expr(:call, :plot, x, ys_expr, tail..., Expr(:kw, :labels, labels_expr))
end


X=first(support(dataset)) : 0.001 : last(support(dataset))
plot(X, pdf.(est,X), xlims= support(dataset), ylims=(0,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,375.813 32.2449,375.813 32.2678,375.813 32.2906,375.813 32.3135,375.813 32.3363,375.813 32.3591,375.813 32.382,375.813 32.4048,375.813 32.4276,375.813 
 32.4505,375.813 32.4733,375.813 32.4962,375.813 32.519,375.813 32.5418,375.813 32.5647,375.813 32.5875,375.813 32.6103,375.813 32.6332,375.813 32.656,375.813 
 32.6788,375.813 32.7017,375.813 32.7245,375.813 32.7474,375.813 32.7702,375.813 32.793,375.813 32.8159,375.813 32.8387,375.813 32.8615,375.813 32.8844,375.813 
 32.9072,375.813 32.9301,375.813 32.9529,375.813 32.9757,375.813 32.9986,375.813 33.0214,375.813 33.0442,375.813 33.0671,375.813 33.0899,375.813 33.1128,375.813 
 33.1356,375.813 33.1584,375.813 33.1813,375.813 33.2041,375.813 33.2269,375.813 33.2498,375.813 33.2726,375.813 33.2955,375.813 33.3183,375.813 33.3411,375.813 
 33.364,375.813 33.3868,375.813 33.4096,375.813 33.4325,375.813 33.4553,375.813 33.4782,375.813 33.501,375.813 33.5238,375.813 33.5467,375.813 33.5695,375.813 
 33.5923,375.813 33.6152,375.813 33.638,375.813 33.6608,375.813 33.6837,375.813 33.7065,375.813 33.7294,375.813 33.7522,375.813 33.775,375.813 33.7979,375.813 
 33.8207,375.813 33.8435,375.813 33.8664,375.813 33.8892,375.813 33.9121,375.813 33.9349,375.813 33.9577,375.813 33.9806,375.813 34.0034,375.813 34.0262,375.813 
 34.0491,375.813 34.0719,375.813 34.0948,375.813 34.1176,375.813 34.1404,375.813 34.1633,375.813 34.1861,375.813 34.2089,375.813 34.2318,375.813 34.2546,375.813 
 34.2775,375.813 34.3003,375.813 34.3231,375.813 34.346,375.813 34.3688,375.813 34.3916,375.813 34.4145,375.813 34.4373,375.813 34.4602,375.813 34.483,375.813 
 34.5058,375.813 34.5287,375.813 34.5515,375.813 34.5743,375.813 34.5972,375.813 34.62,375.813 34.6428,375.813 34.6657,375.813 34.6885,375.813 34.7114,375.813 
 34.7342,375.813 34.757,375.813 34.7799,375.813 34.8027,375.813 34.8255,375.813 34.8484,375.813 34.8712,375.813 34.8941,375.813 34.9169,375.813 34.9397,375.813 
 34.9626,375.813 34.9854,375.813 35.0082,375.813 35.0311,375.813 35.0539,375.813 35.0768,375.813 35.0996,375.813 35.1224,375.813 35.1453,375.813 35.1681,375.813 
 35.1909,375.813 35.2138,375.813 35.2366,375.813 35.2595,375.813 35.2823,375.813 35.3051,375.813 35.328,375.813 35.3508,375.813 35.3736,375.813 35.3965,375.813 
 35.4193,375.813 35.4422,375.813 35.465,375.813 35.4878,375.813 35.5107,375.813 35.5335,375.813 35.5563,375.813 35.5792,375.813 35.602,375.813 35.6248,375.813 
 35.6477,375.813 35.6705,375.813 35.6934,375.813 35.7162,375.813 35.739,375.813 35.7619,375.813 35.7847,375.813 35.8075,375.813 35.8304,375.813 35.8532,375.813 
 35.8761,375.813 35.8989,375.813 35.9217,375.813 35.9446,375.813 35.9674,375.813 35.9902,375.813 36.0131,375.813 36.0359,375.813 36.0588,375.813 36.0816,375.813 
 36.1044,375.813 36.1273,375.813 36.1501,375.813 36.1729,375.813 36.1958,375.813 36.2186,375.813 36.2415,375.813 36.2643,375.813 36.2871,375.813 36.31,375.813 
 36.3328,375.813 36.3556,375.813 36.3785,375.813 36.4013,375.813 36.4242,375.813 36.447,375.813 36.4698,375.813 36.4927,375.813 36.5155,375.813 36.5383,375.813 
 36.5612,375.813 36.584,375.813 36.6068,375.813 36.6297,375.813 36.6525,375.813 36.6754,375.813 36.6982,375.813 36.721,375.813 36.7439,375.813 36.7667,375.813 
 36.7895,375.813 36.8124,375.813 36.8352,375.813 36.8581,375.813 36.8809,375.813 36.9037,375.813 36.9266,375.813 36.9494,375.813 36.9722,375.813 36.9951,375.813 
 37.0179,375.813 37.0408,375.813 37.0636,375.813 37.0864,375.813 37.1093,375.813 37.1321,375.813 37.1549,375.813 37.1778,375.813 37.2006,375.813 37.2235,375.813 
 37.2463,375.813 37.2691,375.813 37.292,375.813 37.3148,375.813 37.3376,375.813 37.3605,375.813 37.3833,375.813 37.4062,375.813 37.429,375.813 3

In [ ]:
histogram(data, nbins=100, xlims=support(dataset))

In [ ]:
dataset = GenerateDatasets.magdon_ismail_and_atiya
data = dataset()
est = NeuralDensityEstimator([1024], -50,50)
fit!(est, data; epochs=10_000)

In [ ]:

X=-.0 : 0.001 : 16.0
plot(X, pdf.(est,X), xlims= support(dataset), ylims=(0,1))

In [ ]:
histogram(X, data, xlims= xlims=(-20,20), ylims=(0,1))

In [ ]:
minimum(data)

In [ ]:
@less dataset()